In [2]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=56565367d16ed1c4fa49da8200c1904a5e30299dad8ff0dcf9ad65fedb111f23
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [20]:
import wikipedia

def fetch_wikipedia_article(title):
    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.PageError:
        print(f"Error: Wikipedia page '{title}' not found.")
        return None

# Example usage
article_title = "earth"
article_text = fetch_wikipedia_article(article_title)
if article_text:
    print(article_text[:500])  # Print first 500 characters of the article


Death is the irreversible cessation of all biological functions that sustain an organism. For organisms with a brain, death can instead be focused on this organ. Brain death is sometimes used as a legal definition of death. The remains of a former organism normally begin to decompose shortly after death. Death inevitably occurs in all organisms eventually. Some organisms, such as Turritopsis dohrnii, are biologically immortal, however they can still die from means other than aging.Determining wh


In [21]:
import re

def clean_text(text):
    # Remove URLs, citations, and other unwanted patterns
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    # Add more cleaning steps as needed
    return text

cleaned_text = clean_text(article_text)
print(cleaned_text[:500])  # Print cleaned text


Death is the irreversible cessation of all biological functions that sustain an organism. For organisms with a brain, death can instead be focused on this organ. Brain death is sometimes used as a legal definition of death. The remains of a former organism normally begin to decompose shortly after death. Death inevitably occurs in all organisms eventually. Some organisms, such as Turritopsis dohrnii, are biologically immortal, however they can still die from means other than aging.Determining wh


In [22]:
def build_vocabulary(text):
    words = text.split()
    vocabulary = set(words)
    word_to_index = {word: idx for idx, word in enumerate(vocabulary)}
    return vocabulary, word_to_index

vocabulary, word_to_index = build_vocabulary(cleaned_text)
print(f"Vocabulary size: {len(vocabulary)}")
print(f"Sample words: {list(vocabulary)[:10]}")


Vocabulary size: 2615
Sample words: ['External', 'assimilated', 'retrospectively.', 'Consciousness', 'Cochem', 'Law', 'planarian.', 'of', 'ideology', 'find']


In [23]:
import numpy as np

def sgd(X_train, y_train, learning_rate=0.01, epochs=10):
    num_samples, num_features = X_train.shape
    weights = np.zeros(num_features)

    for epoch in range(epochs):
        for i in range(num_samples):
            # Forward pass
            prediction = np.dot(X_train[i], weights)

            # Backpropagation
            error = y_train[i] - prediction
            gradient = X_train[i] * error

            # Update weights
            weights += learning_rate * gradient

    return weights

# Example usage
X_train = np.array([[1, 2], [2, 3], [3, 4]])
y_train = np.array([3, 5, 7])
weights = sgd(X_train, y_train)
print("Learned weights:", weights)


Learned weights: [0.7856114  1.13662549]


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform([cleaned_text])
y_train = np.array([0])  # Placeholder target since we're using unsupervised learning

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [25]:
def answer_question(question):
    # Preprocess the question
    cleaned_question = clean_text(question)

    # Concatenate the question and article text
    combined_text = cleaned_question + ' ' + cleaned_text

    # Vectorize the combined text
    combined_vector = vectorizer.transform([combined_text])

    # Predict the most relevant sentence from the article as the answer
    sentence_scores = combined_vector.toarray()[0][1:]  # Skip the first element (question)

    # Debugging: Print the sentence scores
    print("Sentence scores:", sentence_scores)

    # Find the index of the sentence with the highest score
    most_relevant_sentence_index = np.argmax(sentence_scores)

    # Debugging: Print the index of the most relevant sentence
    print("Most relevant sentence index:", most_relevant_sentence_index)

    # Split text into sentences, handling potential empty strings
    sentences = [sentence.strip() for sentence in cleaned_text.split('.') if sentence.strip()]

    # Debugging: Print the number of sentences
    print("Number of sentences:", len(sentences))

    # Get the answer from the selected sentence
    if sentences:
        answer = sentences[300]
    else:
        answer = "Sorry, I couldn't find an answer."

    return answer

# Example usage
question = "find"
answer = answer_question(question)
print("Answer:", answer)


Sentence scores: [1 1 2 ... 1 1 2]
Most relevant sentence index: 1856
Number of sentences: 388
Answer: This process allows the individual's soul to withstand death and transform into life after death


In [39]:
!pip install transformers wikipedia-api
!pip install torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.5 MB/s eta 0:00:00


In [40]:
import wikipediaapi
from transformers import BertForQuestionAnswering, BertTokenizer
import torch


# Load pre-trained BERT model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


def fetch_wikipedia_text(title):
    try:
        wiki = wikipediaapi.Wikipedia(
            user_agent='42 Wikipedia Scraper/1.0',  # User agent must be specified
            language='en',
            extract_format=wikipediaapi.ExtractFormat.WIKI
        )
        page = wiki.page(title)
        if page.exists():
            return page.text
        else:
            print(f"Error: Wikipedia page '{title}' not found.")
            return None
    except Exception as e:
        print(f"Error fetching Wikipedia page: {e}")
        return None

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:

def answer_question(question, text):
    # Tokenize input question and text
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt", truncation=True)

    # Perform question-answering
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract start and end scores from outputs
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Find the tokens with the highest start and end scores
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1  # Add 1 to include the last token

    # Convert token IDs to actual tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())

    # Combine tokens to form the answer
    answer = tokenizer.convert_tokens_to_string(tokens[answer_start:answer_end])

    # Post-process answer (remove special tokens and additional spaces)
    answer = answer.replace("[CLS]", "").replace("[SEP]", "").strip()
    return answer

In [69]:
# Fetch Wikipedia text (replace 'Chatbot' with your desired topic)
article_title = "Phrases from The Hitchhiker's Guide to the Galaxy"
article_text = fetch_wikipedia_text(article_title)

# Example usage
question = "What is the Answer to the Ultimate Question of Life, the Universe, and Everything?"

answer = answer_question(question, article_text)
print(">===============================<")
print("Question:", question)
print("Answer:", answer)
print(">===============================<")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


>===============================<
Question: What is the Answer to the Ultimate Question of Life, the Universe, and Everything?
Answer: 42
>===============================<
